# Entailment-based Zero-Shot Text Classification

This notebook contains the code to run quick Entailment-based Zero-Shot Text Classification experiments, a dataset is provided with it

In [1]:
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from tqdm import tqdm
from transformers import pipeline

## Define your classifier

Select between ```facebook/bart-large-mnli``` and ```roberta-large-mnli```

Just uncomment the code section you want

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# RoBERTa
classifier = pipeline("zero-shot-classification",
                      model="roberta-large-mnli", device=device) # You can set the device to 0 or 1 manually if you want

# BART
# classifier = pipeline("zero-shot-classification",
#                       model="facebook/bart-large-mnli", device=device) # You can set the device to 0 or 1 manually if you want

Using device: cuda


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Import your data

Modify the ```DATAPATH``` to the path of dataset you want to test

In [3]:
DATAPATH = "/data/ARENAS_Automatic_Extremist_Analysis/ARENAS_Automatic_Extremist_Analysis/Entailment/Davidson-test-rayane.csv"
df= pd.read_csv(DATAPATH)

# If you want to replace the class names use the following line
# df['class'] = df['class'].apply(lambda x: x.replace('ok', 'free'))


unique_classes = df['class'].unique()
print("The unique classes in the data are :",unique_classes)

The unique classes in the data are : ['offensive' 'ok' 'hateful']


## Inference

You can modify the hypothesis template ```hypothesis_template``` To your desired format

In [5]:
hypothesis_template = "This text has {} perspective."

sequences_to_classify = list(df['text'])
true_labels = list(df['class'])
candidate_labels = unique_classes
print("The candidate labels are: ", candidate_labels)

predicted_labels = []
for sequence in tqdm(sequences_to_classify, desc="Classifying sequences"):
    classifier_output = classifier(sequence, candidate_labels , hypothesis_template= hypothesis_template)
    predicted_label = classifier_output['labels'][0]
    predicted_labels.append(predicted_label)


The candidate labels are:  ['offensive' 'ok' 'hateful']


Classifying sequences:   0%|          | 10/2477 [00:02<05:23,  7.63it/s]/data/ARENAS_Automatic_Extremist_Analysis/ARENAS_Automatic_Extremist_Analysis/venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Classifying sequences: 100%|██████████| 2477/2477 [03:46<00:00, 10.94it/s]


## Classification metrics

In [6]:
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, labels=candidate_labels, average='macro', zero_division=1)
recall = recall_score(true_labels, predicted_labels, labels=candidate_labels, average='macro', zero_division=1)
f1 = f1_score(true_labels, predicted_labels, labels=candidate_labels, average='macro', zero_division=1)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Macro F1 Score:", f1)

Accuracy: 0.6693580944691159
Precision: 0.46720252786429256
Recall: 0.4596520444939516
Macro F1 Score: 0.4540398325547212


## Support 

If you have any issues or improvement suggestions please contact: rayane.ghilene@ensea.fr